In [3]:
import pandas as pd
import sqlalchemy as sal
from dotenv import load_dotenv
import os
import numpy as np
load_dotenv()

# Create a connection to the database
server_database = os.getenv("SERVER_DATABASE")
engine = sal.create_engine(f'mssql+pyodbc://@{server_database}?trusted_connection=yes&driver=SQL+Server')
conn = engine.connect()

In [4]:
df = pd.read_csv('../data/patient_list_3oct24.csv', parse_dates=['first_date'])
df['first_date'] = df['first_date'].dt.strftime('%Y-%m-%d')

# remove person with person_id = 0, BUG
df = df[df['person_id'] != 0]

### Exclusion criteria:
Drug-diagnosis pair

In [5]:
with open('..\sql\exclusion\drug_diag_pair.sql', 'r') as f:
    sql_q = f.read()
    f.close()

# Due to memory problem, we will query in batch
# We will query 10000 at a time
pt_list = df['person_id'].to_list()
n_per_batch = 5000
n_batches = len(pt_list) // n_per_batch + 1

result=None

for i in range(n_batches):
    print("starting batch ", i+1, " of ", n_batches, "subject ", i*n_per_batch, " to ", (i+1)*n_per_batch - 1)
    pt_sub = str(pt_list[i*n_per_batch:(i+1)*n_per_batch])[1:-1]
    sql_q_sub = sql_q.replace('insert_list_person_id', pt_sub)

    if result is None:
        print("no exisiting dataframe, creating new one")
        result = pd.read_sql(sql_q_sub, conn)
        print("finising batch ", i+1, " of ", n_batches, 'number of records',len(result))
    else:
        next_result = pd.read_sql(sql_q_sub, conn)
        
        result = pd.concat([result, next_result])
        print("finising batch ", i+1, " of ", n_batches, 'number of records',len(result))


starting batch  1  of  36 subject  0  to  4999
no exisiting dataframe, creating new one


In [6]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59937 entries, 0 to 689
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   person_id         59937 non-null  int64 
 1   first_hf_date     14407 non-null  object
 2   first_ckd_date    39328 non-null  object
 3   first_obs_date    19453 non-null  object
 4   glp_start_date    2508 non-null   object
 5   sglt2_start_date  5568 non-null   object
dtypes: int64(1), object(5)
memory usage: 3.2+ MB


In [7]:
result['sglt2_hf'] = result['first_hf_date'] < result['sglt2_start_date']
result['sglt2_ckd'] = result['first_ckd_date'] < result['sglt2_start_date']
result['glp_obs'] = result['first_obs_date'] < result['glp_start_date']

In [8]:
result[['sglt2_hf', 'sglt2_ckd', 'glp_obs']].sum()

sglt2_hf     1339
sglt2_ckd    2915
glp_obs      1402
dtype: int64

In [9]:
df_merged = df.merge(result[['person_id','sglt2_hf', 'sglt2_ckd', 'glp_obs']], on='person_id', how='left')

In [11]:
df_merged.to_pickle("../data/exclude_df_drug_diag_pair_3oct.pkl")


In [12]:
df_merged = pd.read_pickle("../data/exclude_df_drug_diag_pair_3oct.pkl")

In [13]:
sglt2_hf_pt = df_merged.loc[(df_merged['criteria'] == 'drug') & df_merged['sglt2_hf'], 'person_id'].to_list()
sglt2_ckd_pt = df_merged.loc[(df_merged['criteria'] == 'drug') & df_merged['sglt2_ckd'], 'person_id'].to_list()
glp_obs_pt = df_merged.loc[(df_merged['criteria'] == 'drug') & df_merged['glp_obs'], 'person_id'].to_list()

print("pt. with sglt2_hf: ", len(sglt2_hf_pt))
print("pt. with sglt2_ckd: ", len(sglt2_ckd_pt))
print("pt. with glp_obs: ", len(glp_obs_pt))

# union all the person_id together
exc_dx_drug = list(set(sglt2_hf_pt + sglt2_ckd_pt + glp_obs_pt))

print("pt. with sglt2_hf or sglt2_ckd or glp_obs: ", len(exc_dx_drug))


pt. with sglt2_hf:  240
pt. with sglt2_ckd:  255
pt. with glp_obs:  149
pt. with sglt2_hf or sglt2_ckd or glp_obs:  549


### Exclusion criteria:
Age < 18 year

In [15]:
## Get year at birth
sql = """
SELECT person_id, year_of_birth
FROM omop.person
"""

df_yob = pd.read_sql(sql, conn)

In [16]:
df = df.merge(df_yob, on='person_id', how='left')

In [17]:
df['first_date'] = pd.to_datetime(df['first_date'])

# Get age of patient at first diagnosis date
df['age'] = df['first_date'].dt.year - df['year_of_birth']

In [18]:
age18_pt = df.loc[df['age'] < 18, 'person_id'].to_list()

### Exclusion criteria
have T1DM diagnosis

In [19]:
with open("../sql/exclusion/t1dm.sql", "r") as file:
    sql = file.read()
    file.close()

t1dm = pd.read_sql(sql, conn)

In [21]:
t1dm_pt = t1dm['person_id'].to_list()

### Union all excluded patients

In [22]:
print("pt. with age < 18: ", len(age18_pt))
print("pt. with t1dm: ", len(t1dm_pt))
print("pt. with drug_diag: ", len(exc_dx_drug))

all_exclude_pt = set(age18_pt + t1dm_pt + exc_dx_drug)

print("All excluded pt. : ", len(all_exclude_pt))

pt. with age < 18:  1360
pt. with t1dm:  2372
pt. with drug_diag:  549
All excluded pt. :  3827


In [23]:
# remove excluded pt. from the df
df = df[~df['person_id'].isin(all_exclude_pt)]

In [27]:
df.person_id.nunique()

169714

In [25]:
df.to_csv('../data/cohort_3oct.csv', index=False)

In [26]:
df['criteria'].value_counts()

criteria
diag+lab+drug    58052
diag             52667
diag++drug       28116
lab              14532
diag+lab          7340
drug              7114
lab+drug          1893
Name: count, dtype: int64